In [9]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
from scipy import signal

import os
from glob import glob

Folder Paths

In [10]:
greenPath = './ppg_GREEN/'
redPath = './ppg_RED/'
irPath = './ppg_IR/'

BandPass

In [11]:
# BandPass Filter Parameters

sampling_freq = 100 # Hz

lowcut = 0.5                         # Lower Cutoff Frequency in Hz
highcut = 35.0                       # Higher Cutoff Frequency in Hz
nyquist_freq = 0.5 * sampling_freq   # Nyquist frequency in Hz
order = 4                            # Filter order

# Design Bandpass Filter
b, a = signal.butter(order, [lowcut / nyquist_freq, highcut / nyquist_freq], btype='band')

In [16]:
# Average AC Component of Trials
avgAC = 0
numOfFiles = 0

for file in glob(redPath + '*.csv'):
    numOfFiles += 1
    ppg_data = pd.read_csv(file)
    noisy_ppgOne = ppg_data["PPG1"].values
    # Apply BandPass Filter to noisy ECG Signal
    filtered_ppg_1 = signal.filtfilt(b, a, noisy_ppgOne)

    # Baseline Correction
    baseline_ppg_1 = signal.medfilt(filtered_ppg_1, kernel_size=51)  # Apply a median filter as a baseline
    corrected_ppg_1 = filtered_ppg_1 - baseline_ppg_1

    # Smoothing
    smoothed_ppg_1 = signal.savgol_filter(corrected_ppg_1, window_length=51, polyorder=3)

    # Remove the DC Component
    dc_component = np.mean(smoothed_ppg_1)
    ppg_ac= smoothed_ppg_1 - dc_component

    # Calculate the AC Component
    peaks, _ = find_peaks(ppg_ac)  # Adjust height threshold as needed
    troughs, _ = find_peaks(-ppg_ac)  # Detect troughs by inverting signal
    ac_component = ppg_ac[peaks].mean() - ppg_ac[troughs].mean()

    # Add to avg
    avgAC += ac_component

    # Print the AC Component for trial
    print(f'AC Component of {file} : {ac_component}')

print('---------------------------------------------------')
print(f'Average AC Component of Trials using Bandpass is : {avgAC / numOfFiles}')


---------------------------------------------------


ZeroDivisionError: division by zero